# 第4章 ウェブプログラミング


### 本セクションで用いるライブラリの準備
本セクションでは
* HTTPライブラリである[requests](http://requests-docs-ja.readthedocs.io/en/latest/)
* jqueryライクにXML/HTMLを解析する[pyquery](http://pythonhosted.org/pyquery/)
ライブラリを使用します。

ターミナル/コマンドプロンプト上で下記コマンドを実行し、上記ライブラリをインストールしてください。

In [2]:
$ cd data-science-bootcamp
$ pip install requests  # Requestsライブラリ
$ pip install pyquery   # pyqueryライブラリ

SyntaxError: invalid syntax (<ipython-input-2-897dffd8b03b>, line 1)

---

### Q31. wget
requestsライブラリを利用し、[Yahoo!ファイナンス業種別銘柄一覧：サービス業](https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050)のトップページのhtmlを取得し表示せよ。

In [1]:
import requests

target_url = "https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050"
response = requests.get(target_url)
html = response.text
print(html)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ja"> 
<head> 

<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta property="fb:app_id" content="464062600275671">
<meta property="og:site_name" content="Yahoo!ファイナンス">
<meta property="og:type" content="article">
<meta property="og:url" content="https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050">
<meta property="og:image" content="https://s.yimg.jp/images/finance/common/image/ogp.png">
<meta name="keywords" content="株価,チャート,日経平均,銘柄,投資信託,外国為替,ドル,ユーロ,ダウ,ナスダック,Dow,NASDAQ">
<meta http-equiv="Refresh" content="60">

<title>業種別銘柄一覧：サービス業 - Yahoo!ファイナンス</title>
<link rel="stylesheet" href="https://s.yimg.jp/images/finance/common/css/master-finance3.css?date=20180426" type="text/css" media="all">
<link href="https://s.yimg.jp/images

### Q32. スクレピング（1/4）
課題Q31で取得したHTMLを解析して、サービス業を営む上場企業の銘柄コード、およびそれに対応する（ページアクセス時点での）株価のリストを20件を取得せよ。（ヒント）外部ライブラリを使用してもよい（例: pyquery）。

In [2]:
import requests
from pyquery import PyQuery as pq

target_url = "https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050"
response = requests.get(target_url)
html = response.text

stocks = {}
dom = pq(html)
for stock_obj in dom("tr.yjM").items():
    stock_code = int(stock_obj("td.yjM a").text())
    price = float(stock_obj("div.price").text().replace(",", ""))
    stocks[stock_code] = price
    
print(stocks)

{1954: 3250.0, 2120: 905.0, 2121: 3660.0, 2122: 1889.0, 2124: 2234.0, 2127: 3230.0, 2130: 1102.0, 2134: 73.0, 2136: 1335.0, 2137: 990.0, 2139: 762.0, 2144: 245.0, 2146: 3110.0, 2148: 710.0, 2150: 970.0, 2151: 1332.0, 2152: 843.0, 2153: 1714.0, 2154: 3520.0, 2156: 357.0}


### Q33. スクレピング（2/4）
[Yahoo!ファイナンス業種別銘柄一覧：サービス業](https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050)のサイトでは、1ページあたり最大20件の銘柄とその株価が表示される設計になっている。また「次の20件」をクリックすることで更に別の20件の銘柄を確認することができる。当該サイトの[トップページ](https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050)のHTMLを解析し、当該ページに表示された「次の20件」リンクのURLを取得せよ。

### Q34. スクレピング（3/4）
[Yahoo!ファイナンス業種別銘柄一覧：サービス業](https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050)のサイトには、サービス業を営む400以上の企業（以下サービス系企業）の株価情報が掲載されている。上記サイトで閲覧可能なすべてのサービス系企業の株価を取得せよ。ただし、出力は下記のように、（ページアクセス時点での）株価が高い順に、銘柄コード名、株価のペアを表示するものとする。


> (6030, 16730.0)

> (4661, 10515.0)

> (9643, 10150.0)
> ...


In [3]:
import requests
from pyquery import PyQuery as pq
from urllib.parse import urljoin

def get_stock_list(url="https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050"):
    response = requests.get(url)
    html = response.text

    stocks = {}
    dom = pq(html)
    
    # 「次の20件」ページへのリンクを取得
    next_url = None
    for a_obj in dom("div.s150 a").items():
        if a_obj.text() == "次の20件":
            path = a_obj.attr("href")
            next_url = urljoin(url, path)
            break     

    # urlで指定されたページに掲載された株価情報をdict形式で取得
    for stock_obj in dom("tr.yjM").items():
        try:
            stock_code = int(stock_obj("td.yjM a").text())
            price = float(stock_obj("div.price").text().replace(",", ""))
            stocks[stock_code] = price
        except ValueError:
            pass

    return (stocks, next_url)

stock_list = {}
next_url = "https://stocks.finance.yahoo.co.jp/stocks/qi/?ids=9050"
while next_url:
    partial_stock_list, next_url = get_stock_list(next_url)
    for stock_code, price in partial_stock_list.items():
        stock_list[stock_code] = price

sorted_stock_list = sorted(stock_list.items(), key=lambda x: x[1], reverse=True)
for stock_code, price in sorted_stock_list:
    print((stock_code, price))


(6572, 16320.0)
(6030, 15050.0)
(6577, 14560.0)
(4661, 10875.0)
(9643, 10080.0)
(6573, 9310.0)
(9735, 8199.0)
(6080, 7920.0)
(9632, 7000.0)
(6196, 6880.0)
(6563, 6750.0)
(6539, 6700.0)
(2157, 6690.0)
(6028, 6320.0)
(4337, 6240.0)
(9744, 5980.0)
(9661, 5830.0)
(6049, 5800.0)
(4343, 5690.0)
(2331, 5350.0)
(4751, 5270.0)
(4849, 5190.0)
(4324, 5170.0)
(9616, 5130.0)
(6565, 4850.0)
(9671, 4755.0)
(6087, 4670.0)
(9663, 4630.0)
(2497, 4575.0)
(6553, 4440.0)
(5261, 4365.0)
(6569, 4340.0)
(9733, 4275.0)
(4544, 4270.0)
(2175, 4200.0)
(9672, 4195.0)
(6550, 4130.0)
(2429, 4005.0)
(2413, 3990.0)
(9603, 3975.0)
(8920, 3960.0)
(6532, 3900.0)
(9701, 3890.0)
(2195, 3880.0)
(9740, 3865.0)
(6561, 3790.0)
(4801, 3770.0)
(6568, 3760.0)
(4342, 3680.0)
(6184, 3665.0)
(9678, 3665.0)
(2121, 3660.0)
(2154, 3525.0)
(9637, 3515.0)
(2475, 3435.0)
(6050, 3410.0)
(9636, 3405.0)
(9699, 3310.0)
(9639, 3285.0)
(6084, 3275.0)
(9722, 3265.0)
(9731, 3260.0)
(1954, 3250.0)
(2127, 3230.0)
(6198, 3195.0)
(4301, 3180.0)
(6185

### Q35. スクレピング（4/4）
任意の銘柄コードstock_codeが与えられたとき、Yahoo!ファイナンスのウェブサイトをリアルタイムに解析することで、銘柄コードstock_codeに対応する企業名、現在の株価、および前日終値を返す関数get_stock_info(stock_code)を実装せよ。なお、出力は{'company': 企業名, "current_stock_price":現在の株価, "last_close":前日終値}という辞書形式で返すこと。また、出力は存在しない銘柄コードが入力された場合はNoneオブジェクトを出力せよ。

（ヒント）株価詳細ページのURL（例は[コチラ](https://stocks.finance.yahoo.co.jp/stocks/detail/?code=4689)）を観察して、どのようなページを解析対象とするかを決めること。

### Q36. 図書・雑誌検索API（1/5）
[CiNii Books API](https://support.nii.ac.jp/ja/cib/api/b_opensearch)を利用し、「お好み焼き」というキーワードに関する図書・雑誌リストを10件取得し、そのタイトルを出力せよ。

（ヒント） 最低限使用するクエリパラメータは「フリーワードq」と「出力フォーマットformat」の2つ。

In [5]:
import requests
import json

CINII_API_BASE_URL = "http://ci.nii.ac.jp/books/opensearch/search"
query = {"q": "お好み焼き", "format": "json", "count":10}
response = requests.get(CINII_API_BASE_URL, params=query)

json_content = json.loads(response.content)
books = json_content['@graph'][0]['items']

for i, book in enumerate(books):
    print("{id}: {title}".format(id=i+1, title=book['title']))

1: お好み焼き食べたい : 具とソースで満腹バリエ33種
2: 広島ベストスポット
3: 神戸とお好み焼き : まちづくりと比較都市論の視点から
4: お好み焼きたこ焼きもんじゃ焼き&鉄板焼き
5: まぼろしのお好み焼きソース
6: 浜内千波の野菜たっぷりやせる!お好み焼きダイエット
7: お好み焼き90%たこ焼き10%の本 : 粉もん100%レシピ&店ガイド
8: 絶品お好み焼き : たこ焼き・もんじゃ焼き : 人気店の味を家で楽しむ
9: 「街的 (まちてき)」ということ : お好み焼き屋は街の学校だ
10: お好み焼き繁盛店「鶴橋風月」成功のキーワード100


### Q37. 図書・雑誌検索API（2/5）
[CiNii Books API](https://support.nii.ac.jp/ja/cib/api/b_opensearch)を利用し、静岡県の図書館に蔵書されている図書・雑誌のリストを10000件取得し、出版年ごとに図書・雑誌のタイトル数を出力せよ。出力は出版年が若い順に出版年、タイトル数のペアを表示するものとする。また、「19--」のように出版年が一部欠損しているものは出力結果から除外するものとする。

In [7]:
import requests
import json

CINII_API_BASE_URL = "http://ci.nii.ac.jp/books/opensearch/search"
query = {"area": 22, "format": "json", "count":10000}

response = requests.get(CINII_API_BASE_URL, params=query)
json_content = json.loads(response.content)
books = json_content['@graph'][0]['items']
print(len(books))

years = {}
for book in books:
    if 'dc:date' in book:
        pub_year = book['dc:date']
        if not '-' in pub_year:
            pub_year = int(pub_year)
            years.setdefault(pub_year, 0)
            years[pub_year] += 1
            
sorted_years = sorted(years.items(), key=lambda x:x[0])  
for year, title_num in sorted_years:
    print(year, title_num)


1873 1
1875 1
1876 1
1877 1
1878 1
1881 1
1882 1
1883 1
1885 1
1886 1
1887 2
1890 1
1894 2
1895 5
1896 4
1897 1
1898 1
1899 2
1900 1
1901 1
1903 5
1904 2
1905 5
1906 4
1907 1
1908 8
1909 4
1910 8
1911 5
1912 2
1913 3
1914 9
1915 8
1916 5
1917 8
1918 4
1919 9
1920 6
1921 9
1922 7
1923 17
1924 33
1925 20
1926 21
1927 24
1928 25
1929 30
1930 21
1931 19
1932 27
1933 39
1934 29
1935 18
1936 31
1937 35
1938 33
1939 73
1940 96
1941 49
1942 44
1943 59
1944 56
1945 12
1946 20
1947 47
1948 81
1949 64
1950 58
1951 52
1952 67
1953 44
1954 46
1955 69
1956 85
1957 83
1958 81
1959 62
1960 57
1961 41
1962 51
1963 64
1964 97
1965 52
1966 51
1967 45
1968 36
1969 35
1970 28
1971 39
1972 40
1973 28
1974 30
1975 33
1976 35
1977 56
1978 55
1979 47
1980 55
1981 45
1982 65
1983 78
1984 64
1985 50
1986 54
1987 56
1988 55
1989 43
1990 50
1991 54
1992 84
1993 88
1994 196
1995 270
1996 3910
1997 1954
10000


### Q38. 図書・雑誌検索API（3/5）

Q36の課題で実装したコードを改良し、[CiNii Books API](https://support.nii.ac.jp/ja/cib/api/b_opensearch)を利用し、任意のキーワードqueryが与えられたときに、それに関する図書・雑誌リストを10件取得し、そのタイトル、著者名、および書誌詳細ページのURL（例は[コチラ](http://ci.nii.ac.jp/ncid/BB0521742X)）を返す関数simple_search_booksを実装せよ。なお、出力は{"title":タイトル, "author":著者名, "detail_url":書誌詳細ページのURL}という辞書のリストで行うものとする。また、キーワードにマッチする図書・雑誌がなかった場合はNoneオブジェクトを返すように実装せよ。


### Q39. 図書・雑誌検索API（4/5）

CiNii Booksのウェブサイト上の任意の書誌詳細ページのURL（例は[コチラ](http://ci.nii.ac.jp/ncid/BB0521742X)）が与えられたとき、そのページのHTMLを解析することで、該当書誌の概要文（CiNii Booksでは「内容説明」に該当）を取得する関数get_abstractを実装せよ。get_abstractの入力引数はurlとして実装せよ。

例えば、get_abstact("http://ci.nii.ac.jp/ncid/BB0521742X") は以下を返す：

>粉もん発祥の地・神戸には、ソースを作るメーカーが何社もあり、それぞれがお好み焼き用、焼きそば用、たこ焼き用など、たくさんの種類を販売している。それを数種類ブレンドし、かすを入れたのが、長田地区のお好み焼き。人気店「駒」でも同じだが、店で使用するソース会社が経営の危機に陥った。高利貸し、ヤクザ、人情篤い任侠、おまけにＢ級グルメ選手権の地方選抜が絡んで…。


### Q40. 図書・雑誌検索API（5/5）

Q38,39で実装した関数simple_search_booksおよびget_abstractを改良して、任意のキーワードqueryが与えられたときに、それに関する図書・雑誌リストを最大100件取得し、そのタイトル、著者名、書誌詳細ページのURL、および図書・雑誌の概要文を返す関数search_booksを実装せよ。なお、出力は{"title":タイトル, "author":著者名, "detail_url":書誌詳細ページのURL, "abstract":概要文}という辞書のリストで行うものとする。また、キーワードにマッチする図書・雑誌がなかった場合はNoneオブジェクトを返すように実装せよ。